<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Generative Models For Spatial-Based Bottom <br><br> Hypoxia Forecasting In The Black Sea</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

Do not forget to activate the **environnement esa_diffusion**

In [ ]:
# ---------- Librairies ----------
import os
import sys
import dawgz
import wandb
import xarray
import random
import numpy             as np
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# ---------- Jupyter ----------
%matplotlib inline
plt.rcParams.update({'font.size': 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

# Moving to code folder
%cd poseid

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [34]:
dataset_directory = "training/"
dataset_path      = "../../../../../../scratch/acad/bsmfc/victor/data/deep_learning_3D/"

# Extraction of the files name
files = [f for f in os.listdir(dataset_path + dataset_directory) if os.path.isfile(os.path.join(dataset_path + dataset_directory, f))]

# Creating complete path
files = [dataset_path + dataset_directory + f for f in files]

In [35]:
import xarray as xr

data = xr.open_mfdataset(files, engine="netcdf4", combine="by_coords")

In [38]:
data.nbytes / 1e9

22.97644344

In [ ]:
for i in range(50):
    plt.figure(figsize=(10, 10))
    plt.imshow(data["votemper"][i, 0, :, :])
    plt.colorbar()
    plt.show()